In [3]:
import time, cv2
import numpy as np
from threading import Thread
from djitellopy import Tello
from ultralytics import YOLO

In [ ]:
# 드론 연결 및 읽어오기
def videoRecorder(detection_model, tello, frame_read, keepRecording):
    # 동영상 저장을 위한 설정
    height, width, _ = frame_read.frame.shape

    fps_list = []
    frame_count = 0
    avg_fps = 0  # 초기값 설정
    start_time = time.time()

    # 비디오 코덱 설정 및 VideoWriter 객체 생성
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    video = cv2.VideoWriter('video.mp4', fourcc, 30, (width, height))

    try:
        while keepRecording():
            # 프레임 읽기
            frame = frame_read.frame
            if frame is not None:
                #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                # 모델 추론
                pred_results = detection_model(frame, conf=0.25)
                for result in pred_results:
                    boxes = result.boxes
                    x1, y1, x2, y2 = map(int, boxes.xyxy[0])
                    cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 3)
                    label = f"{result.names[int(boxes.cls)]} {boxes.conf[0]:.2f}"
                    cv2.putText(frame, label, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0), 3)

                video.write(frame)
                frame_count += 1

                # 매 30 프레임마다 FPS 계산 및 VideoWriter 업데이트
                if frame_count % 30 == 0:
                    elapsed_time = time.time() - start_time
                    current_fps = frame_count / elapsed_time
                    fps_list.append(current_fps)
                    avg_fps = sum(fps_list) / len(fps_list)

                    # VideoWriter 객체 재생성
                    video.release()
                    video = cv2.VideoWriter('video.mp4', fourcc, avg_fps, (width, height))
                
                # FPS를 frame에 표시
                cv2.putText(frame, f"FPS: {avg_fps:.2f}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

                cv2.imshow("Tello Frame", frame)
            else:
                print("Failed to capture frame")
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    except Exception as e:
        print(f"Error in video recording: {e}")
    finally:
        video.release()
        cv2.destroyAllWindows()
        print("Video recording stopped and saved")


# fruit_detection 모델 불러오기
fruit_model = YOLO("best_model_path")

# Tello 객체 생성
tello = Tello()
keepRecording = True

def should_continue():
    return keepRecording

try:
    # Tello 연결 & 스트리밍 활성화
    tello.connect()
    tello.streamon()
    print("Tello connected and stream started")

    # 약간의 대기 시간 추가
    time.sleep(2)

    # frame 읽기 객체
    frame_read = tello.get_frame_read()

    # 영상 녹화
    recorder = Thread(target=videoRecorder, args=(fruit_model, tello, frame_read, lambda: keepRecording))
    recorder.start()

    # 특정 키를 입력하기 전까지 무한 대기
    while True:
        if input("Press 'q' to quit: ").lower() == 'q':
            keepRecording = False
            break

    # 드론 행동 제어
    #tello.takeoff()     # 드론 이륙
    #tello.move_up(100)  # 100cm 상승
    #tello.land()        # 드론 착륙
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    keepRecording = False
    recorder.join()
    tello.streamoff()
    print("Program ended")

[INFO] tello.py - 129 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 438 - Send command: 'command'
[ERROR] tello.py - 458 - 'utf-8' codec can't decode byte 0xcc in position 0: invalid continuation byte
[INFO] tello.py - 438 - Send command: 'command'
[INFO] tello.py - 462 - Response command: 'ok'
[INFO] tello.py - 438 - Send command: 'streamon'
[INFO] tello.py - 462 - Response streamon: 'ok'


Tello connected and stream started

0: 480x640 (no detections), 131.8ms
Speed: 4.4ms preprocess, 131.8ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)
Error in video recording: index 0 is out of bounds for dimension 0 with size 0
Video recording stopped and saved


left block unavailable for requested intra mode
error while decoding MB 0 29, bytestream 2453
left block unavailable for requested intra mode
error while decoding MB 0 25, bytestream 5543
error while decoding MB 47 41, bytestream -6
left block unavailable for requested intra mode
error while decoding MB 0 15, bytestream 9211
left block unavailable for requested intra mode
error while decoding MB 0 27, bytestream 3464
left block unavailable for requested intra mode
error while decoding MB 0 20, bytestream 7811
left block unavailable for requested intra mode
error while decoding MB 0 25, bytestream 6481
[INFO] tello.py - 438 - Send command: 'streamoff'
[INFO] tello.py - 462 - Response streamoff: 'ok'


Program ended
